In [ ]:
# Databricks notebook source
# =============================================================================
# 🚀 AUTOMATED PRODUCTION SERVING ENDPOINT DEPLOYMENT (UPDATED & OPTIMIZED)
# =============================================================================

%pip install xgboost
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput,
    ServedEntityInput
)
import mlflow
from mlflow.tracking import MlflowClient
import time, os, sys


print("=" * 80)
print("🚀 STARTING PROD ENDPOINT DEPLOYMENT")
print("=" * 80)


# =============================================================================
# CONFIGURATION
# =============================================================================
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"
PROD_ALIAS = "production"

WORKLOAD_SIZE = "Small"
SCALE_TO_ZERO = True


# =============================================================================
# INITIALIZE CLIENTS
# =============================================================================
try:
    w = WorkspaceClient()
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()

    print("✅ WorkspaceClient initialized")
    print("✅ MLflow client initialized\n")

except Exception as e:
    print("❌ Failed to initialize clients:", e)
    sys.exit(1)


# =============================================================================
# 1️⃣ DETECT LATEST EXPERIMENT → MODEL TYPE → MODEL NAME
# =============================================================================
def detect_latest_model_info():
    print("🔍 Detecting latest experiment & model ...")

    experiments = client.search_experiments()
    latest_exp = max(experiments, key=lambda e: e.last_update_time)

    exp_name = latest_exp.name.lower()

    if "xgboost" in exp_name:
        model_type = "xgboost"
    elif "rf" in exp_name or "randomforest" in exp_name:
        model_type = "rf"
    elif "linear" in exp_name:
        model_type = "linear"
    else:
        model_type = "generic"

    model_name = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_{model_type}_uc"
    endpoint_name = f"house-price-{model_type}-prod"

    print(f"✅ Experiment: {latest_exp.name}")
    print(f"✅ Model type detected: {model_type}")
    print(f"✅ Model name: {model_name}")
    print(f"✅ Endpoint name: {endpoint_name}\n")

    return model_name, endpoint_name


# =============================================================================
# 2️⃣ GET PRODUCTION MODEL VERSION
# =============================================================================
def get_prod_version(model_name):
    print("🔍 Fetching production model version ...")

    versions = client.search_model_versions(f"name='{model_name}'")

    for v in versions:
        mv = client.get_model_version(model_name, v.version)
        if PROD_ALIAS in mv.aliases:
            print(f"✅ Production Version: {mv.version}")
            print(f"   Run ID: {mv.run_id}")
            return mv.version

    print("❌ Production alias not found. Run staging→prod promotion first.")
    return None


# =============================================================================
# 3️⃣ CHECK IF ENDPOINT EXISTS
# =============================================================================
def endpoint_exists(endpoint_name):
    print(f"🔍 Checking endpoint: {endpoint_name}")

    try:
        eps = w.serving_endpoints.list()

        for ep in eps:
            if ep.name == endpoint_name:
                print("✅ Endpoint exists\n")
                return True
        print("ℹ️ Endpoint does NOT exist\n")
        return False

    except Exception as e:
        print("⚠️ Error checking endpoint:", e)
        return False


# =============================================================================
# 4️⃣ CREATE / UPDATE ENDPOINT
# =============================================================================
def deploy(endpoint_name, model_name, version, exists):

    served = ServedEntityInput(
        entity_name=model_name,
        entity_version=version,
        workload_size=WORKLOAD_SIZE,
        scale_to_zero_enabled=SCALE_TO_ZERO
    )

    print("=" * 80)
    print("📦 DEPLOYMENT CONFIG")
    print("=" * 80)
    print(f"Endpoint: {endpoint_name}")
    print(f"Model: {model_name}")
    print(f"Version: {version}")
    print(f"Workload: {WORKLOAD_SIZE}")
    print(f"Scale-to-zero: {SCALE_TO_ZERO}\n")

    try:
        if exists:
            print("🔄 Updating endpoint ...")
            w.serving_endpoints.update_config(
                name=endpoint_name,
                served_entities=[served]
            )
            print("✅ Update initiated\n")

        else:
            print("🆕 Creating endpoint ...")
            cfg = EndpointCoreConfigInput(
                name=endpoint_name,
                served_entities=[served]
            )
            w.serving_endpoints.create(
                name=endpoint_name,
                config=cfg
            )
            print("✅ Creation initiated\n")

        return True

    except Exception as e:
        print("❌ Deployment failed:", e)
        return False


# =============================================================================
# 5️⃣ WAIT UNTIL READY
# =============================================================================
def wait_ready(endpoint_name):

    print("⏳ Waiting for endpoint to be ready...\n")

    timeout = 1200
    step = 15
    waited = 0

    while waited < timeout:
        ep = w.serving_endpoints.get(name=endpoint_name)

        state = ep.state
        update = str(state.config_update)
        ready = str(state.ready)

        if "NOT_UPDATING" in update and "READY" in ready:
            print("\n✅ Endpoint is READY!\n")
            return True

        if "FAILED" in update:
            print("❌ Endpoint FAILED:", update)
            return False

        print(f"⏳ {waited}s — State: {update}, Ready: {ready}")
        time.sleep(step)
        waited += step

    print("❌ Timeout - endpoint not ready")
    return False


# =============================================================================
# MAIN FLOW
# =============================================================================
if __name__ == "__main__":

    model_name, endpoint_name = detect_latest_model_info()

    version = get_prod_version(model_name)
    if not version:
        sys.exit(1)

    exists = endpoint_exists(endpoint_name)

    if not deploy(endpoint_name, model_name, version, exists):
        sys.exit(1)

    if not wait_ready(endpoint_name):
        sys.exit(1)

    print("✅✅ Production endpoint deployment completed successfully!")

    try:
        dbutils.notebook.exit("ENDPOINT_READY")
    except:
        pass
